In [1]:
%matplotlib inline

In [2]:
import tempfile
from io import StringIO
import jovian
import tabula
import camelot
import pandas as pd
import numpy as np
from PIL import Image, ImageFont, ImageDraw
from pdf2image import convert_from_path
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import dataframe_image as dfi
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

<IPython.core.display.Javascript object>

In [3]:
# conta o número de páginas do pdf
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import resolve1

PDF = '2020-04_Importações.pdf'
file = open(PDF, 'rb')
parser = PDFParser(file)
document = PDFDocument(parser)
Quant_paginas= resolve1(document.catalog['Pages'])['Count']


In [4]:
#extrai os dados de cada pagina do pdf e mescla depois em um só dataframe
df = pd.DataFrame()
df_2 = pd.DataFrame()
df_apoio= df = pd.DataFrame()
merged_df= pd.DataFrame()
new= pd.DataFrame()

Quant_paginas=Quant_paginas


for page in range(0,Quant_paginas):  
    df_apoio=df_2
    df = (camelot
              .read_pdf(PDF,
                        flavor="stream",
                        suppress_stdout=True,
                        table_areas=['5,818,579,50'],
                        pages=f'{page}')
              [0].df)

        
    new = df[6].str.split("\n", n = 3, expand = True)
    if (len(new.columns)>1):
     df[13]=df[11]
     df[14]=df[12]
     df[12]=df[10]
     df[11]=df[9]
     df[10]=df[8]
     df[9]=df[7]
     df[6]= new[0]
     df[7]= new[1]
     df[8]= new[2]
    
   #cria 3 colunas de apoio para separar a tabela em 3
    row=df.loc[(df[0] =="") & (df[2] == "") & (df[3] == "") & (df[4] == "") & (df[1] != "") & (df[1]!= df[6])].index
    df.loc[row,'Tipo']= df[1]
    row=df.loc[(df[5] =="") & (df[9] == "")  & (df[6] != "") & (df[11] != df[6])].index
    df.loc[row,'Tipo2']= df[6]
    row=df.loc[(df[10] =="") & (df[12] == "") & (df[13] == "") & (df[14] == "")  & (df[11] != "") &  (df[11] != df[6])].index
    df.loc[row,'Tipo3']= df[11]

    df['Tipo'] = df['Tipo'].ffill(axis=0)
    df.loc[1,'Tipo2']= df['Tipo'].iloc[-1]
    df['Tipo2'] = df['Tipo2'].ffill(axis=0)
    df.loc[1,'Tipo3']= df['Tipo2'].iloc[-1]
    df['Tipo3'] = df['Tipo3'].ffill(axis=0)
       
  
    # Seleciona as colunas referente a tabela da primeira parte do PDP
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df1= df.drop([ 5,6,7,8,9,10,11,12,13,14, 'Tipo2', 'Tipo3'], axis = 1)
    df2 = df1.rename(columns = {0:"Origem", 1:"Quant mensal", 2:"Valor mensal",3:"Quant anual",4:"Valor anual"})
    
    # Seleciona as colunas referente a tabela da segunda parte do PDP
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    df3= df.drop([0,1,2,3,4,10,11,12,13,14, 'Tipo', 'Tipo3'], axis = 1)
    df4 = df3.rename(columns = {5:"Origem", 6:"Quant mensal", 7:"Valor mensal", 8:"Quant anual",9:"Valor anual", 'Tipo2':'Tipo'})
    
    # Seleciona as colunas referente a tabela da terceira parte do PDP
    df5 = pd.DataFrame()
    df6 = pd.DataFrame()
    df5= df.drop([0,1,2,3,4,5,6,7,8,9,'Tipo', 'Tipo2'], axis = 1)
    df6 = df5.rename(columns = {10:"Origem", 11:"Quant mensal", 12:"Valor mensal", 13:"Quant anual",14:"Valor anual", 'Tipo3':'Tipo'})
    
    #Mescla as três partes do PDP em um só dataframe
    merged_df = pd.concat([df2, df4, df6],ignore_index=True)
     
    #concatena a tabela atual com a tabela salva do pdf anterior
    df_2= pd.concat([merged_df, df_apoio], ignore_index=True)   
  


In [5]:
df_2.head(20)

,Origem,Quant mensal,Valor mensal,Quant anual,Valor anual,Tipo
0,,,,,,NaN
1,,,,,,NaN
2,,CUSTOMS IMPORT DECLARATION,,,,NaN
3,,OIL COUNTRY TUBULAR GOODS,,,,NaN
4,COUNTRY,MO,MO,YEAR,YEAR,NaN
5,OF,QNTY,CIF,QNTY,CIF,NaN
6,ORIGIN,TONS,$,TONS,$,NaN
7,,,,,,NaN
8,,,7306218050,,,NaN
9,,,,,,NaN


In [6]:
#confere o final do dataframe
df_2.to_excel(r'D:\Reforma\Dados_tubos3.xlsx', index = False)
